In [35]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
from math import log
import heapq
import json
import requests
from nltk.corpus import wordnet as wn
import os
import gensim 
from gensim.models import Word2Vec
nltk.download('punkt')
import sqlite3


In [ ]:
#create the sentence list from the db for the embedded model
os.chdir('/media/removable/sdcard/CompTools/computational-tools-2018')

conn = sqlite3.connect('./backend/parser.sqlite3')
c = conn.cursor()
sentences=[]
x=c.execute('Select title,tokenized_plot From movies where tokenized_plot is not NULL;')
for movie in x:
    print((movie[1]))
    sentences.append(movie[1].split(","))


In [57]:
#create CBOW model
model1 = gensim.models.Word2Vec(sentences, min_count = 1,  
                              size = 100, window = 5) 

#create skip-grams model
model2 = gensim.models.Word2Vec(sentences, min_count = 1, size = 100, 
                                             window = 5, sg = 1) 


In [59]:
#save the models to files
os.chdir("/media/removable/sdcard/CompTools")
model1.wv.save_word2vec_format('model1.bin')
 
model2.wv.save_word2vec_format('model2.bin')

In [ ]:
# creating synonyms from inputs    

inputArgs = sys.argv
synonymList=[list() for arg in sys.argv]

#grab synonyms from API
for position in range(len(sys.argv)):
    r = requests.get('https://od-api.oxforddictionaries.com:443/api/v1/entries/en/' + synonymList[position]+ '/synonyms', headers = {'app_id': 'a76c92b5', 'app_key': '107c3736bfdc71a084306ecb73aafa26'})
    synonyms=json.loads(r.text)
    for i in synonyms["results"]:
        for j in i["lexicalEntries"]:
            for k in j["entries"]:
                for v in k["senses"]:
                    for w in v["synonyms"]:
                        synonymList[position].append(w["text"])

#load the desired model
model1 = Word2Vec.load('model1.bin')

#find synonyms from embedded model
for i in range(len(synonymList)):
    for word in word2vec.wv.most_similar(inputArgs[i]):
        synonymList[i].append(word[0])

#eliminate any duplicate synonyms
for each in synonymList:
    each=set([word for word in each if len(word.split()) == 1])

    
#-------------datamuse api--------------------------------------------------------------------    
# for i in range(len(synonymList)) in synonymList:
#     response=requests.get("https://api.datamuse.com/words?ml=",inputArgs[i],"&max=5")
#     first_syns=json.loads(response.text)
#     for first_syn in first_syns:
#          second_syns=wordnet.synsets(first_syn)
#             for second_syn in second_syns:
#                 third_syns=pyhugeconnector.thesaurus_entry(
#                 word=item, 
#                 api_key='e2e7d8c39bbda388d13c8dbcbb25533b', 
#                 pos_tag='n', 
#                 ngram=1, 
#                 relationship_type='syn') 
#                 for third_syn in third_syns:
#                     synonymList[i].append(third_syn)
#     synonymList[i]=set(synonymList[i])
#---------------------------------------------------------------------------------------------



#--------------pyhugeconnector api------------------------------------------------------------

# for word_dict in jsontext:
#     synlist.append(word_dict['word'])

# for item in range(len(inputArgs)):
#     synonymList.append(sys.argv[item+1])
#     nltkSyns = wordnet.synsets(sys.argv[item+1]) 
#     for firstSyn in nltkSyns:
#         wordSynonyms=pyhugeconnector.thesaurus_entry(
#             word=item, 
#             api_key='e2e7d8c39bbda388d13c8dbcbb25533b', 
#             pos_tag='n', 
#             ngram=1, 
#             relationship_type='syn') 
        #Use ‘syn’ for synonyms, ‘ant’ for antonyms, ‘rel’ for related terms, ‘sim’ 
        #for similar terms,‘usr’ for user suggestions and None for all (default: None)
        
#         for secondSyn in wordSynonyms:
#             synonymList[item].append(secondSyn)
#     synonymList[item]=set(synonymList[item])
#----------------------------------------------------------------------------------------------





#use the bloom filter here


#grab db entries for synonym info
flat_synonyms=','.join(item for sublist in synonymList for item in sublist)

db=sqlite3.connect('./backend/parser.sqlite3')
cursor=db.cursor()
sqlCall=db.execute("SELECT word,movie_ids from word_movies where word in ("+flat_synonyms+")") 

#make dictionary from the sql call
wordDict={}
for entry in sqlCall:
    wordDict[entry[0]]=entry[1].split(",")

#loop through the dictionary and sum the movies
movieScores={}
for group in range(len(synonymList)):
    for word in synonymList[group]:
        if movieScores.get(movie):
            movieScores[word][group]+=1
        else:
            movieScores[word]=[0,0,0]
            movieScores[word][group]=1    
    
for movie in movieScores:
    original_syn_count=0
    for word in inputArgs:
        if movie in wordDict[word]:
            original_syn_count+=1
    resultDictionary[movie]=math.log(movieScores[movie][0]+0.00001, 1.5)+math.log(movieScores[movie][1]+0.00001, 1.5)+math.log(movieScores[movie][2]+0.00001, 1.5)+5*original_syn_count
    
recommendation=heapq.nlargest(10,resultDictionary.values())
print("Based on your inputs, we recommend the following:",'\n')
for movie in recommendation:
    print(movie)

